In [26]:
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import os
import itertools

from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from sklearn.utils import shuffle

In [27]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="char-embeddings.txt",
               word2vec_output_file="gensim_char-embeddings.txt")

In [28]:
char_embeddings = KeyedVectors.load_word2vec_format("../gensim_char-embeddings.txt", binary=False)
char_vectors = char_embeddings.wv

In [29]:
glove_model = KeyedVectors.load_word2vec_format("../gensim_glove_vectors25.txt", binary=False)
vectors = glove_model.wv

In [143]:
def glove_char_embeddings(tokens):
    chars_num = 150;
    dim = 300
    
    arr = np.zeros(dim)
    
    for word in tokens:
        for char in word:
            try:
                arr = np.vstack((arr, char_vectors[char]))
            except:
                pass
    
    arr = arr[1:, :]
    
    while arr.shape[0] < chars_num:
        arr = np.vstack((arr, np.zeros(dim)))
        
    arr = arr.reshape(1, chars_num, dim)
    
    return np.array(arr)

In [87]:
def glove_vects(tokens):
    
    arr = np.zeros(25)
    
    for tok in tokens:
        try:
            arr = np.add(arr, vectors[tok])
        except:
            pass
        
    return np.array(arr)

In [83]:
def preprocessTweet(tweet):
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(tweet)
    
    return glove_char_embeddings(tokens)

In [147]:
def load_document(data_location, htf):
    tweets = []
    labels = []
    
    for line in open(os.path.join(data_location, htf)).readlines():
        line_split = line.strip().split('\t')
        tweets.append(line_split[1])
        labels.append(np.array([int(line_split[2])]))

    Y = np.array(labels)
    
    X_vects = np.zeros(45000)
    X_vects = X_vects.reshape(1, 150, 300)
    
    for tweet in tweets:
        prep = preprocessTweet(tweet)
        X_vects = np.vstack((X_vects, prep))
        
#     X_vects = [preprocessTweet(tweet, htf) for tweet in tweets]

    return {'X_vects': X_vects[1:], 'Y': Y}

In [121]:
def create_data(data_location):
    ht_files = sorted(os.listdir(data_location))

    Xs = []
    ys = []
    ht_list = []
    for htf in ht_files:
        ht_dict = load_document(data_location, htf)

        ht_list.append(htf)
        ys.append(ht_dict['Y'])
        Xs.append(ht_dict['X_vects'])

    return Xs, ys, ht_list

In [153]:
Xs, ys, ht_list = create_data('../trial_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])

x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

# x_train, y_train = shuffle(x_train, y_train)

# print(x_train.shape)
# print(y_train.shape)
# print(x_test.shape)
# print(y_test.shape)

max_features = 1024
timesteps = 150
dimensions = 300

model = Sequential()
# model.add(Embedding(input_dim=25, output_dim=256))
model.add(LSTM(128, input_shape = (timesteps, dimensions), return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

ValueError: Error when checking target: expected dropout_59 to have 3 dimensions, but got array with shape (550, 1)

In [81]:
Xs, ys, ht_list = create_data('../train_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])
x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

model = Sequential()
model.add(Dense(64, input_dim=25, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

class_weight = {0: 1., 1: 8.3}

model.fit(x_train, y_train,
          epochs=1000,
          batch_size=128,
         class_weight=class_weight)

score = model.evaluate(x_test, y_test, batch_size=1)
# print(score)
y_pred = model.predict_classes(x_test, verbose=1)

print(np.hstack((y_test, y_pred)))

Epoch 1/1000
11186/11186 [==============================] - 3s 312us/step - loss: 1.8283 - acc: 0.4933
Epoch 2/1000
11186/11186 [==============================] - 0s 12us/step - loss: 1.4048 - acc: 0.6304
Epoch 3/1000
11186/11186 [==============================] - 0s 11us/step - loss: 1.2682 - acc: 0.6883
Epoch 4/1000
11186/11186 [==============================] - 0s 12us/step - loss: 1.1957 - acc: 0.7311
Epoch 5/1000
 4480/11186 [===========>..................] - ETA: 0s - loss: 1.1640 - acc: 0.7650

KeyboardInterrupt: 

In [63]:
Xs, ys, ht_list = create_data('../train_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])
x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

max_features = 1024

model = Sequential()
# model.add(Embedding(input_dim=25, output_dim=256))
model.add(LSTM(128, input_shape = (len_of_seq, 25), return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)

y_pred = model.predict_classes(x_test, verbose=1)
print(y_pred)
# score = model.evaluate(x_test, y_test, batch_size=16)

ValueError: Input 0 is incompatible with layer lstm_16: expected ndim=3, found ndim=2

In [54]:
Xs, ys, ht_list = create_data('../train_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])
x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)
y_pred = regressor.predict(x_test)
print(np.hstack((y_test, y_pred)))

ValueError: Error when checking input: expected lstm_5_input to have 3 dimensions, but got array with shape (11186, 25)

In [168]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 300
timesteps = 150
num_classes = 2

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Xs, ys, ht_list = create_data('../trial_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])

x_train, y_train = shuffle(x_train, y_train)

x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

model.fit(x_train, y_train,
          batch_size=64, epochs=15)

print(np.hstack((y_test, model.predict(x_test))))

Epoch 1/15
550/550 [==============================] - 16s 30ms/step - loss: 0.6649 - acc: 0.9200
Epoch 2/15
550/550 [==============================] - 4s 7ms/step - loss: 0.4807 - acc: 0.9273
Epoch 3/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2768 - acc: 0.9273
Epoch 4/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2703 - acc: 0.9273
Epoch 5/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2607 - acc: 0.9273
Epoch 6/15
550/550 [==============================] - 4s 6ms/step - loss: 0.2612 - acc: 0.9273
Epoch 7/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2611 - acc: 0.9273
Epoch 8/15
550/550 [==============================] - 4s 6ms/step - loss: 0.2608 - acc: 0.9273
Epoch 9/15
550/550 [==============================] - 3s 6ms/step - loss: 0.2613 - acc: 0.9273
Epoch 10/15
550/550 [==============================] - 3s 6ms/step - loss: 0.2615 - acc: 0.9273
Epoch 11/15
550/550 [==========================